# Student Loan Risk with Deep Learning

In [2]:
pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
    --------------------------------------- 7.3/376.0 MB 41.2 MB/s eta 0:00:09
   -- ------------------------------------- 21.5/376.0 MB 54.4 MB/s eta 0:00:07
   --- ------------------------------------ 34.1/376.0 MB 57.0 MB/s eta 0:00:06
   ----- ---------------------------------- 48.2/376.0 MB 59.1 MB/s eta 0:00:06
   ------ --------------------------------- 59.0/376.0 MB 59.7 MB/s eta 0:00:06
   ------- -------------------------------- 72.9/376.0 MB 59.6 MB/s eta 0:00:06
   --------- ------------------------------ 87.0/376.0 MB 61.0 MB/s eta 0:00:05
   ---------- ----------------------------- 99.4/376.0 MB 61.0 MB/s eta 0:00:05
   ----------- --------------------------- 110.9/376.0 MB 60.5 MB/s eta 0:00:05
   ------------ -------------------------- 123.5/376.0 MB 60.2 MB/s eta 0:00:05
   -------------- ------------------------ 137.6/376.0 MB 60.6 MB/s eta 0:00:04
   --------------- ----------------------- 149.9/3

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking

x = loans_df.drop(columns=["credit_ranking"])
# Review the features DataFrame
x.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

# Fit the scaler to the features training dataset


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
input_features = x_train_scaled.shape[1]

# Review the number of features
print(f"Number of features: {input_features}")


Number of features: 11


In [14]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 8

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 4

# Define the number of neurons in the output layer
output_neurons = 1


In [15]:
# Create the Sequential model instance
nn_model = Sequential()


# Add the first hidden layer
nn_model.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=input_features))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_nodes_layer2, activation="relu"))


# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(Dense(units=1, activation="sigmoid"))


c:\Users\sokhna.thiaw\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Display the Sequential model summary
nn_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137 (548.00 B)

 Trainable params: 137 (548.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [17]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(x_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5277 - loss: 0.6993 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5850 - loss: 0.6624 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6396 - loss: 0.6375 
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6478 - loss: 0.6336
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6861 - loss: 0.6198
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7092 - loss: 0.6028
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7236 - loss: 0.5877 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7149 - loss: 0.5886 
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7224 - loss: 0.5766 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7374 - loss: 0.5723
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7460 - loss: 0.5503 
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.73

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [20]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

model_loss, model_accuracy = nn_model.evaluate(x_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.4f}, Accuracy: {model_accuracy:.4f}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7574 - loss: 0.4987  
Loss: 0.5299, Accuracy: 0.7325


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [22]:
# Set the model's file path
nn_model.save("student_loans.keras")


# Export your model to a keras file
loaded_model = tf.keras.models.load_model("student_loans.keras")


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [26]:
# Set the model's file path


# Load the model to a new object
loaded_model = tf.keras.models.load_model("student_loans.keras")


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [29]:
# Make predictions with the test data
predictions = loaded_model.predict(x_test_scaled)


# Display a sample of the predictions
print(predictions[:5])


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[0.3117888 ]
 [0.30102202]
 [0.61154103]
 [0.7557659 ]
 [0.9813109 ]]


In [32]:
# Save the predictions to a DataFrame and round the predictions to binary results
predicted_labels = (predictions > 0.5).astype(int)
predicted_df = pd.DataFrame(predicted_labels, columns=["Predicted"])
predicted_df.head()



,Predicted
0,0
1,0
2,1
3,1
4,1


### Step 4: Display a classification report with the y test data and predictions

In [33]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predicted_labels))


              precision    recall  f1-score   support

           0       0.70      0.77      0.73       188
           1       0.77      0.70      0.74       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.74      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



Q1: What data would you collect?
To build a recommendation system for student loan options, I would collect:

Academic information (GPA, major, degree level)

School ranking and tuition cost

Employment status/income

Loan history and repayment status

Credit score and financial literacy level

Personal preferences (repayment period, loan amount)

Why?
This data helps tailor loan options to each student’s financial needs, ability to repay, and preferences, making the recommendations more effective.

Q2: What filtering method and why?
Method: Content-based filtering

Why?
Because recommendations would be based on each student's profile and preferences (not peer behavior), which aligns well with content-based filtering.

Q3: Real-world challenges
Data Privacy and Consent:
Financial and academic data is sensitive. Obtaining and securing consent is legally and ethically necessary.

Bias and Fairness:
The model could unintentionally discriminate against students from certain backgrounds if trained on biased data. Ensuring fairness and inclusivity is critical.